<a href="https://colab.research.google.com/github/mowillia/phantom_pen/blob/master/corpora_metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Metadata for the Corpora

This IPython notebook computes metadata for the short stories and articles in our corpora


In [1]:
# Packages

import pandas as pd
import math, scipy
import copy
import re
import os
import time

# used in the count of words
import string

import nltk.data # natural language tool kit

# for tokenizing sentences according by the words
from nltk.tokenize import sent_tokenize, word_tokenize # $ pip install nltk
nltk.download('punkt')

import numpy as np

from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


##### 

### Getting List of Files

In [0]:
'''
    For the given path, get the List of all files in the directory tree 
    From: https://thispointer.com/python-how-to-get-list-of-files-in-directory-and-sub-directories/
'''
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [5]:
# start time
start_time = time.time()

# directory name
dirname = '/content/drive/My Drive/writrly_proj_files/Atlantic_essays/' 

# getting raw list of files
raw_file_list = getListOfFiles(dirname)

# eliminating the .DS Store files
raw_file_list = [x for x in raw_file_list if not ('.DS_Store' in x)];
   
# directory name
dirname_stories = '/content/drive/My Drive/writrly_proj_files/short_stories/' 

# getting raw list of files
raw_file_list_stories = getListOfFiles(dirname_stories)

# eliminating the .DS Store files
raw_file_list_stories = [x for x in raw_file_list_stories if not ('.DS_Store' in x)];


print('Run Time:', str(time.time()-start_time), ' sec')

Run Time: 3.5314478874206543  sec


In [6]:
# number of essays 
len(raw_file_list+raw_file_list_stories)

2174

In [7]:
## Test code to collect Atlantic article meta data

k = 1678

filename = raw_file_list[k]

#locations of forward slashes
sl_locs = [i for i, letter in enumerate(filename) if letter == '/']
# location of slash before subject
idx_subj_1 =  sl_locs[5]
idx_subj_2 = sl_locs[6]
# location of last forward slash
idx = sl_locs[-1]

# concatenated file name
filename_concat = filename[idx+1:]

# first underscore after title and before author
idx1 = filename[idx+1:].find('_')
# location of break between author and date
idx2 = filename_concat[idx1+1:].find('_')


# metadata
#author = filename[]
topic = filename[idx_subj_1+1:idx_subj_2]
title = filename_concat[:idx1]
author = filename_concat[idx1+1:][:idx2]
date = filename_concat[idx1+1:][idx2+1:-4]

print(topic)
print(title)
print(author)

print(date)


international
How the (Once) Most Corrupt Country in the World Got Clean(er)  
Jonah Blank
May 2, 2019


In [8]:
## Test code to collect short story meta data

k = 100

filename = raw_file_list_stories[k]

#locations of forward slashes
sl_locs = [i for i, letter in enumerate(filename) if letter == '/']
# location of slash before subject
idx_subj_1 =  sl_locs[4]
idx_subj_2 = sl_locs[5]
# location of last forward slash
idx = sl_locs[-1]

# concatenated file name
filename_concat = filename[idx+1:]

# first underscore after title and before author
idx1 = filename[idx+1:].find('_')
# location of break between author and date
idx2 = filename_concat[idx1+1:].find('_')


# metadata
#author = filename[]
topic = filename[idx_subj_1+1:idx_subj_2]
title = filename_concat[:idx1]
author = filename_concat[idx1+1:][idx2+1:-4]

print(topic)
print(title)
print(author)


short_stories
berties-christmas-eve
hh-munro-saki


In [0]:
## Function to collect metadata for Atlantic Articles

def filename_metadata(filename):
  
    '''
    Takes in a file name (containing an article) 
    and outputs the title of the article, the author, and the topic
    
    '''

    #locations of forward slashes
    sl_locs = [i for i, letter in enumerate(filename) if letter == '/']
    # location of slash before subject
    idx_subj_1 =  sl_locs[5]
    idx_subj_2 = sl_locs[6]
    # location of last forward slash
    idx = sl_locs[-1]

    # concatenated file name
    filename_concat = filename[idx+1:]

    # first underscore after title and before author
    idx1 = filename[idx+1:].find('_')
    # location of break between author and date
    idx2 = filename_concat[idx1+1:].find('_')


    # metadata
    #author = filename[]
    topic = filename[idx_subj_1+1:idx_subj_2]
    title = filename_concat[:idx1]
    author = filename_concat[idx1+1:][:idx2]
    date = filename_concat[idx1+1:][idx2+1:-4]

    return {'Topic': topic, 
    'Title': title,
    'Author': author}

In [10]:
# testing function
filename_metadata(raw_file_list[60])

{'Author': 'Amanda Mull',
 'Title': 'I Gave Up and Let Instagram Shop for Me ',
 'Topic': 'technology'}

In [0]:
## Function to collect metadata from short stories

def filename_metadata_stories(filename):

  
    '''
    Takes in a file name (containing a short story) 
    and outputs the title of the article, the author, and the topic
    
    '''
    
    #locations of forward slashes
    sl_locs = [i for i, letter in enumerate(filename) if letter == '/']
    # location of slash before subject
    idx_subj_1 =  sl_locs[4]
    idx_subj_2 = sl_locs[5]
    # location of last forward slash
    idx = sl_locs[-1]

    # concatenated file name
    filename_concat = filename[idx+1:]

    # first underscore after title and before author
    idx1 = filename[idx+1:].find('_')
    # location of break between author and date
    idx2 = filename_concat[idx1+1:].find('_')


    # metadata
    #topic = filename[idx_subj_1+1:idx_subj_2]
    topic = 'short story'
    title = filename_concat[:idx1]
    author = filename_concat[idx1+1:][idx2+1:-4]
    
    # remove hyphen and capitalize author
    author = author.replace('-', ' ')
    author = author.title()
    
    # remove hyphen and capitalize title
    title = title.replace('-', ' ')
    title = title.title()
    
    return {'Topic': topic, 
    'Title': title,
    'Author': author}

In [12]:
#testing function
filename_metadata_stories(raw_file_list_stories[60])

{'Author': 'Anton Chekhov', 'Title': 'A Chameleon', 'Topic': 'short story'}

In [0]:
# create master data frame of corpora
corpora_metadata = pd.DataFrame(columns = ['Topic', 'Title','Author'])

## Fill in elements from the file 
for elem in raw_file_list:
    corpora_metadata = corpora_metadata.append(filename_metadata(elem)
                                       ,ignore_index = True)
for elem in raw_file_list_stories:
    corpora_metadata = corpora_metadata.append(filename_metadata_stories(elem)
                                       ,ignore_index = True)

In [0]:
### saving corpora meta data as a csv file [already saved]
corpora_metadata.to_csv('corpora_metadata.csv')

### Meta Data Calculations

- Computes the number of short stories and short story authors

- Computes the number of articles and authors per category of Atlantic article

In [25]:
## number of short stories in the corpus
len(corpora_metadata[corpora_metadata['Topic']=='short story'])

160

In [26]:
## number of short story authors
story_author = corpus_metadata[corpus_metadata['Topic']=='short story']['Author']
len(set(story_author.tolist()))

52

In [27]:
# number of articles for each topic
topics = set(corpus_metadata['Topic'].tolist())
for topic in topics:
  print(topic, ":", len(corpus_metadata[corpus_metadata['Topic']==topic]))
  
  
  topic_author = corpus_metadata[corpus_metadata['Topic']==topic]['Author']
  print('    ', "num authors:", len(set(topic_author.tolist())))

technology : 229
     num authors: 57
health : 207
     num authors: 60
short story : 160
     num authors: 52
science : 257
     num authors: 59
entertainment : 234
     num authors: 68
international : 142
     num authors: 75
ideas : 252
     num authors: 123
business : 205
     num authors: 68
education : 255
     num authors: 63
politics : 233
     num authors: 52
